In [3]:
import sys
sys.path.append('/home/jupyter/bastianberger/inria1358')
import matplotlib.pyplot as plt
from PIL import Image

In [9]:
import numpy as np

In [11]:
im = Image.open('/home/jupyter/bastianberger/inria1358/raw_data/patches512/train/images/austin28.tif__4__1.png')
np.array(im).shape

(512, 512, 3)

In [4]:
make_patches(source_path='/home/jupyter/bastianberger/inria1358/raw_data/AerialImageDataset',
             save_path='/home/jupyter/bastianberger/inria1358/raw_data/patches500',
             image_size=(500,500))


Checking local data...
0 Patches found for X_train
0 Patches found for y_train
0 Patches found for X_test
❗ Train folders contain 0 images
❗ Test folders contain 0 images

Iterating through folder: train/images


100%|██████████| 180/180 [2:36:43<00:00, 52.24s/it]  



Iterating through folder: train/gt


100%|██████████| 180/180 [06:40<00:00,  2.22s/it]



Iterating through folder: test/images


 78%|███████▊  | 140/180 [2:00:20<34:22, 51.57s/it]  


KeyboardInterrupt: 

In [ ]:
from PIL import Image

In [ ]:
def make_512(source_path: str, save_path: str, image_size=(200,200), image_type="png", max_files = None):
    """Function to create patches from the original images, in order to be able to feed them to the model
        Args:
            source_path: path to the original dataset
            save_path: path where the patches should be saved
            image_size: (width, height) of patches
            image_type: file type that the patches should be outputted in
            max_files: maximum number of images to process per subfolder (selected randomly), None = no limit
    """

    # Checking if directories exist, if not, create them
    for folder in ['train','test']:
        subfolders = ['gt', 'images'] if folder == "train" else ['images']
        for subfolder in subfolders:
            load_path = f'{source_path}/{folder}/{subfolder}'
            save_path_n = f'{save_path}/{folder}/{subfolder}'
            # Check whether the specified path exists or not
            isExist = os.path.exists(save_path_n)
            if not isExist:
                os.makedirs(save_path_n)

    # Checking the local data
    sets = []
    print(Fore.BLUE + "\nChecking local data..." + Style.RESET_ALL)

    # Get # of images from save path
    train_images = len(os.listdir(f"{save_path}/train/images"))
    train_gt = len(os.listdir(f"{save_path}/train/gt"))
    test_images = len(os.listdir(f"{save_path}/test/images"))

    print(f"{train_images} Patches found for X_train")
    print(f"{train_gt} Patches found for y_train")
    print(f"{test_images} Patches found for X_test")

    # Check if >0 images are there for each set (meaning we have data) otherwise add it to the sets that we want to patch
    if train_images == 0 or train_gt == 0:
        print("❗ Train folders contain 0 images")
        sets.append("train")
    else:
        print("ℹ️ Train patches already exist")

    if test_images == 0:
        print("❗ Test folders contain 0 images")
        sets.append("test")
    else:
        print("ℹ️ Test patches already exist")


    # Iterate through the sets
    for set in sets:

        # Subfolders to iterate through, if the set is train, iterate also iterate through GT folder
        subfolders = ['images']
        if set == 'train':
            subfolders.append('gt')

        # Iterate through each subfolder

        for subfolder in subfolders:
            load_path = f'{source_path}/{set}/{subfolder}'
            save_path_n = f'{save_path}/{set}/{subfolder}'

            print(Fore.BLUE + f"\nIterating through folder: {set}/{subfolder}" + Style.RESET_ALL)

            dimensions = image_size if subfolder == "gt" else (image_size[0],image_size[1],3)
            file_count = 0

            files = os.listdir(load_path)

            if max_files is not None:
                files = random.sample(files, max_files)

            for filename in tqdm(files):
                im = Image.open(f'{load_path}/{filename}')
                im = im.resize((512,512))
                im.save(f'{save_path_n}/{filename}.{image_type}'

    # Final Output
    # Get # of images from save path
    train_images = len(os.listdir(f"{save_path}/train/images"))
    train_gt = len(os.listdir(f"{save_path}/train/gt"))
    test_images = len(os.listdir(f"{save_path}/test/images"))

    if len(sets) > 0:
        print(Fore.BLUE + f"\nCounting local images..." + Style.RESET_ALL)
        print(f"{train_images} Patches found for X_train")
        print(f"{train_gt} Patches found for y_train")
        print(f"{test_images} Patches found for X_test")
    print("✅ Patches loaded")

In [ ]:
make_512(source_path='/home/jupyter/bastianberger/inria1358/raw_data/patches500',
         save_path='/home/jupyter/bastianberger/inria1358/raw_data/patches512')

In [1]:
from PIL import Image
import numpy as np
from patchify import patchify
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import tensorflow as tf
from colorama import Fore, Style
from multiprocessing import Pool
source_path='/home/jupyter/bastianberger/inria1358/raw_data/AerialImageDataset'
save_path='/home/jupyter/bastianberger/inria1358/raw_data/test'
image_size=(500,500)
image_type='png'
max_files=None

# Checking if directories exist, if not, create them
for folder in ['train','test']:
    subfolders = ['gt', 'images'] if folder == "train" else ['images']
    for subfolder in subfolders:
        load_path = f'{source_path}/{folder}/{subfolder}'
        save_path_n = f'{save_path}/{folder}/{subfolder}'
        # Check whether the specified path exists or not
        isExist = os.path.exists(save_path_n)
        if not isExist:
            os.makedirs(save_path_n)

# Checking the local data
sets = []

# Get # of images from save path
train_images = len(os.listdir(f"{save_path}/train/images"))
train_gt = len(os.listdir(f"{save_path}/train/gt"))
test_images = len(os.listdir(f"{save_path}/test/images"))

print(f"{train_images} Patches found for X_train")
print(f"{train_gt} Patches found for y_train")
print(f"{test_images} Patches found for X_test")

# Check if >0 images are there for each set (meaning we have data) otherwise add it to the sets that we want to patch
if train_images == 0 or train_gt == 0:
    print("❗ Train folders contain 0 images")
    sets.append("train")
else:
    print("ℹ️ Train patches already exist")

if test_images == 0:
    print("❗ Test folders contain 0 images")
    sets.append("test")
else:
    print("ℹ️ Test patches already exist")


# Iterate through the sets
for set in sets:

    # Subfolders to iterate through, if the set is train, iterate also iterate through GT folder
    subfolders = ['images']
    if set == 'train':
        subfolders.append('gt')

    # Iterate through each subfolder

    for subfolder in subfolders:
        load_path = f'{source_path}/{set}/{subfolder}'
        save_path_n = f'{save_path}/{set}/{subfolder}'

        dimensions = image_size if subfolder == "gt" else (image_size[0],image_size[1],3)
        file_count = 0

        files = os.listdir(load_path)

        if max_files is not None:
            files = random.sample(files, max_files)

        def process_patchify(filename, load_path, save_path_n, subfolder, image_type, dimensions):
            im = Image.open(f'{load_path}/{filename}')
            imarray = np.array(im)

            # Assuming patchify function is imported
            patches = patchify(imarray, dimensions, step=500)

            for i, row in enumerate(patches):
                for j, col in enumerate(row):
                    if subfolder == "gt":
                        im = Image.fromarray(col)
                    else:
                        im = Image.fromarray(col[0])

                    im.save(f'{save_path_n}/{filename}__{i}__{j}.{image_type}')
            
        def wrapper_function(filename):
            process_patchify(filename, load_path, save_path_n, subfolder, image_type, dimensions)

        # ... (Your code before this)

        if __name__ == '__main__':
            pool = Pool()
            for _ in tqdm(pool.imap_unordered(wrapper_function, files), total=len(files)):
                pass

# Final Output
# Get # of images from save path
train_images = len(os.listdir(f"{save_path}/train/images"))
train_gt = len(os.listdir(f"{save_path}/train/gt"))
test_images = len(os.listdir(f"{save_path}/test/images"))

if len(sets) > 0:
    print(f"{train_images} Patches found for X_train")
    print(f"{train_gt} Patches found for y_train")
    print(f"{test_images} Patches found for X_test")
print("✅ Patches loaded")

2023-09-25 09:09:11.196797: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 09:09:16.030216: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2023-09-25 09:09:16.031237: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/

18000 Patches found for X_train
18000 Patches found for y_train
18000 Patches found for X_test
ℹ️ Train patches already exist
ℹ️ Test patches already exist
✅ Patches loaded


In [2]:
source_path='/home/jupyter/bastianberger/inria1358/raw_data/test'
save_path='/home/jupyter/bastianberger/inria1358/raw_data/patches512'
image_size=(500,500)
image_type='png'
max_files=None
from PIL import Image
import numpy as np
from patchify import patchify
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import tensorflow as tf
from colorama import Fore, Style
from multiprocessing import Pool

# Checking if directories exist, if not, create them
for folder in ['train','test']:
    subfolders = ['gt', 'images'] if folder == "train" else ['images']
    for subfolder in subfolders:
        load_path = f'{source_path}/{folder}/{subfolder}'
        save_path_n = f'{save_path}/{folder}/{subfolder}'
        # Check whether the specified path exists or not
        isExist = os.path.exists(save_path_n)
        if not isExist:
            os.makedirs(save_path_n)

# Checking the local data
sets = []
print(Fore.BLUE + "\nChecking local data..." + Style.RESET_ALL)

# Get # of images from save path
train_images = len(os.listdir(f"{save_path}/train/images"))
train_gt = len(os.listdir(f"{save_path}/train/gt"))
test_images = len(os.listdir(f"{save_path}/test/images"))

print(f"{train_images} Patches found for X_train")
print(f"{train_gt} Patches found for y_train")
print(f"{test_images} Patches found for X_test")

# Check if >0 images are there for each set (meaning we have data) otherwise add it to the sets that we want to patch
if train_images == 0 or train_gt == 0:
    print("❗ Train folders contain 0 images")
    sets.append("train")
else:
    print("ℹ️ Train patches already exist")

if test_images == 0:
    print("❗ Test folders contain 0 images")
    sets.append("test")
else:
    print("ℹ️ Test patches already exist")


# Iterate through the sets
for set in sets:

    # Subfolders to iterate through, if the set is train, iterate also iterate through GT folder
    subfolders = ['images']
    if set == 'train':
        subfolders.append('gt')

    # Iterate through each subfolder

    for subfolder in subfolders:
        load_path = f'{source_path}/{set}/{subfolder}'
        save_path_n = f'{save_path}/{set}/{subfolder}'

        print(Fore.BLUE + f"\nIterating through folder: {set}/{subfolder}" + Style.RESET_ALL)

        dimensions = image_size if subfolder == "gt" else (image_size[0],image_size[1],3)
        file_count = 0

        files = os.listdir(load_path)

        if max_files is not None:
            files = random.sample(files, max_files)

        def wrapper_function2(filename):
            im = Image.open(f'{load_path}/{filename}')
            im = im.resize((512,512))
            im.save(f'{save_path_n}/{filename}')

        if __name__ == '__main__':
            pool = Pool()
            pool.map(wrapper_function2, files)

# Final Output
# Get # of images from save path
train_images = len(os.listdir(f"{save_path}/train/images"))
train_gt = len(os.listdir(f"{save_path}/train/gt"))
test_images = len(os.listdir(f"{save_path}/test/images"))

if len(sets) > 0:
    print(Fore.BLUE + f"\nCounting local images..." + Style.RESET_ALL)
    print(f"{train_images} Patches found for X_train")
    print(f"{train_gt} Patches found for y_train")
    print(f"{test_images} Patches found for X_test")
print("✅ Patches loaded")


Checking local data...
18000 Patches found for X_train
18000 Patches found for y_train
18000 Patches found for X_test
ℹ️ Train patches already exist
ℹ️ Test patches already exist
✅ Patches loaded
